In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as ss

In [2]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', None)

In [3]:
# #get rid of scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
# failed sound check 1957441

# Fish

In [5]:
# import file
data = pd.read_csv('data_exp_27347-v3_task-d7fe.csv', sep = ",")
# data_2 = pd.read_csv("pilot_2/data_exp_24005-v8_task-lhx6.csv", sep = ",")


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data.task.unique()

array([nan, 'fishing_control', 'mood_rating_control'], dtype=object)

In [7]:
data = data.replace('null', np.nan)
data  = data.replace('nan', np.nan)


In [8]:
data['start_time'] = round(data['start_time'].astype(float))

In [9]:
data = data[['Participant Private ID', 'UTC Timestamp','key pressed', 'price_displayed', 'environment', 'task', 'function id',
       'builtin rt', 'end_time', 'start_time', 'response', 'round_instance']]


In [10]:
data.rename(columns={"Participant Private ID": "Subject"}, inplace = True)


In [11]:
data = data.sort_values(by = ['Subject', 'start_time'])

In [12]:
data['trial_label'] = data[['environment', 'round_instance']].apply(lambda x: ' '.join(x.astype(str)), axis=1)


In [13]:
# exclude
# data = data[data.Subject != 1972447]

In [14]:
data["environment_binary"] = np.where(data["environment"] == 'pre_mood', 0, 1)

In [15]:
data["mood"] = data["task"].map({"mood_rating_pos":"pos",
                                    "mood_rating_neg":"neg"})

In [16]:
data = data.sort_values(by = ["Subject", "UTC Timestamp"])

In [17]:
data.reset_index(inplace = True)

In [18]:
data.Subject.nunique()

30

# Latency

In [19]:
fish = data[(data.task == "fishing_pos")|(data.task == "fishing_neg")|(data.task == "fishing_control")]



In [20]:
fish = fish[fish['start_time'].notna()]

In [21]:
fish['start_time'] = fish['start_time'].astype(int)

In [22]:
fish.environment.unique()

array(['pre_mood', 'post_mood'], dtype=object)

In [23]:
d = fish.groupby(['Subject','environment_binary', 'round_instance'])['start_time'].diff().shift(-1).reset_index(name = 'latency')
d = d.set_index('index')


In [24]:
fish['latency'] = d['latency']
fish = fish.sort_index().sort_values(['Subject', 'start_time'])

In [25]:
def mask_first(x):
    """
    get rid of the first spike by creating a mask of 0, 1, 1, 1, ...
    """
    result = np.ones_like(x)
    result[0] = 0
    return result

In [26]:
mask = fish.groupby(['Subject', 'environment'])['round_instance'].transform(mask_first).astype(bool)

# data_useful = data_useful.loc[mask]


In [27]:
fish = fish.loc[mask]

In [28]:
fish.head()
fish["mood"] = fish["task"].map({'fishing_pos': "pos", 'fishing_neg':'neg', 'fishing_control':'control'})


In [29]:
fish.groupby("mood")["Subject"].nunique()

mood
control    30
Name: Subject, dtype: int64

In [30]:
fish['mood'].unique()

array(['control'], dtype=object)

In [31]:
fish.latency.isnull().values.any()
fish = fish[fish.latency.notnull()]

In [32]:
# mad
fish.shape

(57955, 17)

In [33]:
median = np.median(fish.latency)
low = median - 3*ss.median_absolute_deviation(fish.latency)
above = median + 3*ss.median_absolute_deviation(fish.latency)
fish = fish[(fish.latency>=low)&(fish.latency<=above)]

In [34]:
# mad
fish.shape

(54682, 17)

# Rating

In [35]:
data.task.unique()

array([nan, 'fishing_control', 'mood_rating_control'], dtype=object)

In [36]:
# response
rating = data[(data.task == "mood_rating_pos")|(data.task == "mood_rating_neg")|(data.task == "mood_rating_control")]


In [37]:
rating = rating[rating['response'].notna()]

# Concat 

In [38]:
rating["latency"] = np.nan*len(rating)

In [39]:
data_complete = pd.concat([rating, fish])


In [40]:
data_complete = data_complete.sort_values(by = "index")

# Backfilling

In [41]:
data_complete["response"].bfill(inplace = True)

# df.bfill(axis ='rows')


In [42]:
data_complete["block"] = data_complete["round_instance"].map({
    1:"1st",
    2:"1st",
    3:"1st",
    4:"1st",
    5:"2nd",
    6:"2nd",
    7:"2nd",
    8:"2nd",
    9:"3rd",
    10:"3rd",
    11:"3rd",
    12:"3rd"
})

In [43]:
data_complete.to_csv("data_control_oct_1_latency_backfilled_happiness.csv")


In [44]:
data.price_displayed.unique()

array([nan, 1.8, 0.7, 0.1, 2.5, 1.2, 0.3])

In [45]:
data.head(100)

,index,Subject,UTC Timestamp,key pressed,price_displayed,environment,task,function id,builtin rt,end_time,start_time,response,round_instance,trial_label,environment_binary,mood
0,0,2035451.000,1601568572170.000,nan,nan,pre_mood,NaN,caller,nan,nan,nan,nan,nan,pre_mood nan,0,NaN
1,1,2035451.000,1601568572232.000,nan,nan,pre_mood,NaN,caller,nan,nan,nan,nan,nan,pre_mood nan,0,NaN
2,2,2035451.000,1601568572232.000,nan,nan,pre_mood,NaN,caller,nan,nan,nan,nan,nan,pre_mood nan,0,NaN
3,3,2035451.000,1601568572232.000,nan,nan,pre_mood,NaN,caller,nan,nan,nan,nan,nan,pre_mood nan,0,NaN
4,4,2035451.000,1601568573064.000,40.000,1.800,pre_mood,fishing_control,fishing_control,868.680,1601568571521.000,1601568570652.000,nan,1.000,pre_mood 1.0,0,NaN
5,5,2035451.000,1601568573206.000,40.000,1.800,pre_mood,fishing_control,fishing_control,4.690,1601568571642.000,1601568571637.000,nan,1.000,pre_mood 1.0,0,NaN
6,6,2035451.000,1601568573331.000,40.000,1.800,pre_mood,fishing_control,fishing_control,23.780,1601568571779.000,1601568571754.000,nan,1.000,pre_mood 1.0,0,NaN
7,7,2035451.000,1601568573473.000,40.000,1.800,pre_mood,fishing_control,fishing_control,28.605,1601568571920.000,1601568571890.000,nan,1.000,pre_mood 1.0,0,NaN
8,8,2035451.000,1601568573598.000,40.000,1.800,pre_mood,fishing_control,fishing_control,27.755,1601568572060.000,1601568572031.000,nan,1.000,pre_mood 1.0,0,NaN
9,9,2035451.000,1601568573754.000,40.000,1.800,pre_mood,fishing_control,fishing_control,40.985,1601568572210.000,1601568572168.000,nan,1.000,pre_mood 1.0,0,NaN
